# Import library

In [238]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np

# Load Dataset

In [194]:
wisata = pd.read_json("dataset.json")
wisata.head()

,id,name,category,location,image,rating,description,coordinate
0,1,Kawah Putih,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-159888094008...,4.8,Danau kawah vulkanik dengan air berwarna putih...,"{'latitude': -7.1655, 'longitude': 107.3999}"
1,2,Tangkuban Perahu,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-158866821440...,4.7,Gunung berapi aktif dengan kawah besar berbent...,"{'latitude': -6.7708, 'longitude': 107.6005}"
2,3,Farmhouse Lembang,Wisata Keluarga,"Lembang, Bandung",https://images.unsplash.com/photo-160056675235...,4.6,Taman rekreasi bertema pedesaan Eropa dengan r...,"{'latitude': -6.8117, 'longitude': 107.6175}"
3,4,Saung Angklung Udjo,Budaya,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151628044061...,4.9,Pusat budaya Sunda yang menampilkan pertunjuka...,"{'latitude': -6.9175, 'longitude': 107.6191}"
4,5,Kampung Daun,Kuliner,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151724813546...,4.5,Restoran alam dengan konsep gazebo-gazebo di a...,"{'latitude': -6.8506, 'longitude': 107.6339}"


# Preprocessing Data

In [195]:
wisata[['latitude', 'longitude']] = pd.json_normalize(wisata['coordinate'])
wisata.head()

,id,name,category,location,image,rating,description,coordinate,latitude,longitude
0,1,Kawah Putih,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-159888094008...,4.8,Danau kawah vulkanik dengan air berwarna putih...,"{'latitude': -7.1655, 'longitude': 107.3999}",-7.1655,107.3999
1,2,Tangkuban Perahu,Alam,"Bandung, Jawa Barat",https://images.unsplash.com/photo-158866821440...,4.7,Gunung berapi aktif dengan kawah besar berbent...,"{'latitude': -6.7708, 'longitude': 107.6005}",-6.7708,107.6005
2,3,Farmhouse Lembang,Wisata Keluarga,"Lembang, Bandung",https://images.unsplash.com/photo-160056675235...,4.6,Taman rekreasi bertema pedesaan Eropa dengan r...,"{'latitude': -6.8117, 'longitude': 107.6175}",-6.8117,107.6175
3,4,Saung Angklung Udjo,Budaya,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151628044061...,4.9,Pusat budaya Sunda yang menampilkan pertunjuka...,"{'latitude': -6.9175, 'longitude': 107.6191}",-6.9175,107.6191
4,5,Kampung Daun,Kuliner,"Bandung, Jawa Barat",https://images.unsplash.com/photo-151724813546...,4.5,Restoran alam dengan konsep gazebo-gazebo di a...,"{'latitude': -6.8506, 'longitude': 107.6339}",-6.8506,107.6339


In [196]:
wisata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           100 non-null    int64  
 1   name         100 non-null    object 
 2   category     100 non-null    object 
 3   location     100 non-null    object 
 4   image        100 non-null    object 
 5   rating       100 non-null    float64
 6   description  100 non-null    object 
 7   coordinate   100 non-null    object 
 8   latitude     100 non-null    float64
 9   longitude    100 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 7.9+ KB


In [197]:
wisata.describe()

,id,rating,latitude,longitude
count,100.000000,100.000000,100.000000,100.000000
mean,50.500000,4.565000,-6.927086,107.563906
std,29.011492,0.155294,0.278158,0.518100
min,1.000000,4.200000,-7.733300,106.546600
25%,25.750000,4.475000,-6.957775,107.302700
50%,50.500000,4.600000,-6.897050,107.617350
75%,75.250000,4.700000,-6.733975,107.666100
max,100.000000,4.900000,-6.556200,108.663400


In [198]:
wisata.isna().sum()

id             0
name           0
category       0
location       0
image          0
rating         0
description    0
coordinate     0
latitude       0
longitude      0
dtype: int64

In [199]:
wisata = wisata.drop(columns=["coordinate"])

In [200]:
print(f"Jumlah data duplikat : {wisata.duplicated().sum()}")

Jumlah data duplikat : 0


In [201]:
wisata[wisata["category"] == "Taman Kota"]

,id,name,category,location,image,rating,description,latitude,longitude
61,62,Taman Balai Kota Bandung,Taman Kota,"Bandung, Jawa Barat",https://images.unsplash.com/photo-161667629452...,4.4,"Taman kota yang sejuk dengan air mancur, area ...",-6.9147,107.6098
80,81,Alun-Alun Kota Garut,Taman Kota,"Garut, Jawa Barat",https://images.unsplash.com/photo-161870125829...,4.3,"Taman kota ikonik dengan air mancur, arena ber...",-7.2101,107.9086


In [202]:
wisata.loc[wisata["category"] == "Taman Alam", "category"] = "Taman"
wisata.loc[wisata["category"] == "Taman Botani", "category"] = "Taman"
wisata.loc[wisata["name"] == "Alun-Alun Bandung", "category"] = "Taman Kota"

In [203]:
wisata["category"].unique()

array(['Alam', 'Wisata Keluarga', 'Budaya', 'Kuliner',
       'Pemandian Air Panas', 'Pantai', 'Taman Kota', 'Kebun Binatang',
       'Pegunungan', 'Taman Hiburan', 'Taman', 'Sejarah', 'Danau'],
      dtype=object)

In [204]:
wisata[wisata["category"] == "Pemandian Air Panas"]

,id,name,category,location,image,rating,description,latitude,longitude
9,10,Pemandian Air Panas Alam Sari Ater,Pemandian Air Panas,"Subang, Jawa Barat",https://images.unsplash.com/photo-158133720487...,4.4,Kolam air panas alami dengan fasilitas lengkap...,-6.5608,107.7589
83,84,Sumber Air Panas Cibolang,Pemandian Air Panas,"Pangalengan, Bandung",https://images.unsplash.com/photo-162180151073...,4.5,Pemandian air panas alami dengan latar pegunun...,-7.1542,107.5861


In [207]:
import json
# Load data tempat
with open('dataset.json', 'r') as f:
    places = json.load(f)

In [208]:
import json
import requests
import time

# Ganti dengan nama file kamu
with open('dataset.json', 'r') as f:
    places = json.load(f)

for place in places:
    full_address = f"{place['name']}, {place['location']}"
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': full_address,
        'format': 'json',
        'limit': 1
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; MyProject/1.0)'  # Penting!
    }

    response = requests.get(url, params=params, headers=headers)
    data = response.json()

    if data:
        place['coordinate']['latitude'] = float(data[0]['lat'])
        place['coordinate']['longitude'] = float(data[0]['lon'])
        print(f"Updated: {place['name']} => {data[0]['lat']}, {data[0]['lon']}")
    else:
        print(f"Gagal cari lokasi: {place['name']}")

    time.sleep(1)  # biar gak di-rate limit

# Simpan file baru
with open('dataset_updated.json', 'w') as f:
    json.dump(places, f, indent=2)

print("Selesai! Lihat file 'places_updated.json'")


Updated: Kawah Putih => -7.16611815, 107.40226688508221
Updated: Tangkuban Perahu => -6.7742798, 107.6361233
Updated: Farmhouse Lembang => -6.8327478, 107.6052985
Updated: Saung Angklung Udjo => -6.898269450000001, 107.65529705969041
Updated: Kampung Daun => -6.816085, 107.589551
Updated: Tebing Keraton => -6.8348584, 107.6638952
Gagal cari lokasi: Situ Patenggang
Gagal cari lokasi: Ranca Upas
Gagal cari lokasi: Kebun Begonia Lembang
Gagal cari lokasi: Pemandian Air Panas Alam Sari Ater
Gagal cari lokasi: Curug Maribaya
Updated: Taman Hutan Raya Ir. H. Juanda => -6.8411686, 107.65037816503335
Updated: Stone Garden => -6.8247319, 107.4381247
Gagal cari lokasi: Gunung Pancar
Updated: Taman Safari Indonesia => -6.7201709, 106.95195277895067
Updated: Kebun Raya Bogor => -6.5983032, 106.79941871332014
Updated: Puncak Bogor => -6.4759723, 106.5990915
Updated: Taman Bunga Nusantara => -6.7277478, 107.0794088
Updated: Pantai Pangandaran => -7.68905865, 108.64359043619461
Updated: Green Canyon 

In [215]:
new_wisata = pd.read_json("dataset_updated.json")
new_wisata["category"].unique()

array(['Alam', 'Wisata Keluarga', 'Budaya', 'Kuliner',
       'Pemandian Air Panas', 'Pantai', 'Kebun Binatang', 'Taman Kota',
       'Pegunungan', 'Taman Hiburan', 'Taman Alam', 'Taman Botani',
       'Sejarah', 'Danau'], dtype=object)

In [217]:
new_wisata.loc[new_wisata["category"] == "Taman Alam", "category"] = "Taman"
new_wisata.loc[new_wisata["category"] == "Taman Botani", "category"] = "Taman"
new_wisata.loc[new_wisata["name"] == "Alun-Alun Bandung", "category"] = "Taman Kota"

In [219]:
new_wisata["category"].unique()

array(['Alam', 'Wisata Keluarga', 'Budaya', 'Kuliner',
       'Pemandian Air Panas', 'Pantai', 'Taman Kota', 'Kebun Binatang',
       'Pegunungan', 'Taman Hiburan', 'Taman', 'Sejarah', 'Danau'],
      dtype=object)

# CBF System


In [222]:
# Gabungkan fitur teks
new_wisata['combined_features'] = new_wisata['category'] + ' ' + new_wisata['location'] + ' ' + new_wisata['description']

In [239]:
# Split data latih dan uji (misalnya 80-20)
train_df, test_df = train_test_split(new_wisata, test_size=0.2, random_state=42)

In [225]:
# Ambil stopwords Bahasa Indonesia
factory = StopWordRemoverFactory()
stopwords_id = factory.get_stop_words()

In [240]:
# TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_id)
X_train = vectorizer.fit_transform(train_df['combined_features'])
X_test = vectorizer.transform(test_df['combined_features'])

In [242]:
# Fit KNN dengan data latih
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X_train)

NearestNeighbors(metric='cosine')

In [243]:
# Contoh rekomendasi dari data test
def recommend_from_test(index):
    test_vector = X_test[index]
    distances, indices = knn.kneighbors(test_vector, n_neighbors=5)
    return train_df.iloc[indices[0]][['name', 'category', 'location', 'rating']]

In [244]:
# Misalnya kita ambil rekomendasi berdasarkan item ke-0 dari test set
print("Rekomendasi untuk:", test_df.iloc[0]['name'])
print(recommend_from_test(0))

Rekomendasi untuk: Sumber Air Panas Cibolang
                                  name             category  \
9   Pemandian Air Panas Alam Sari Ater  Pemandian Air Panas   
13                       Gunung Pancar                 Alam   
65                   Taman Bougenville                Taman   
16                        Puncak Bogor                 Alam   
74                         Curug Sawer                 Alam   

               location  rating  
9    Subang, Jawa Barat     4.4  
13        Sentul, Bogor     4.5  
65  Bandung, Jawa Barat     4.5  
16    Bogor, Jawa Barat     4.6  
74    Garut, Jawa Barat     4.4  


In [212]:
def rekomendasi(preferensi_nama=None, preferensi_kategori=None, df=None, tfidf_matrix=tfidf_matrix, vectorizer=None, top_n=5):
    """
    preferensi_nama: list nama tempat favorit (misal: ["Kawah Putih", "Tangkuban Perahu"])
    preferensi_kategori: list kategori favorit (misal: ["Alam", "Pantai"])
    df: DataFrame wisata
    tfidf_matrix: TF-IDF matrix untuk df["combined_text"]
    vectorizer: objek TfidfVectorizer yang sudah difit
    top_n: jumlah rekomendasi yang ingin dikembalikan
    """
    
    preferensi_vektor = []
    
    # Ambil vektor dari nama tempat
    if preferensi_nama:
        idx_nama = df[df["name"].isin(preferensi_nama)].index.tolist()
        if idx_nama:
            # Konversi ke array dense dengan .toarray() sebelum menghitung mean
            vektor_nama = tfidf_matrix[idx_nama].toarray()
            preferensi_vektor.append(vektor_nama)
    
    # Ambil vektor dari input kategori
    if preferensi_kategori:
        gabung_kategori = " ".join(preferensi_kategori)
        vektor_kategori = vectorizer.transform([gabung_kategori]).toarray()  # Konversi ke array dense
        preferensi_vektor.append(vektor_kategori)
    
    if not preferensi_vektor:
        return "Masukkan minimal satu preferensi: nama tempat dan/atau kategori."
    
    # Gabungkan seluruh vektor dengan stack terlebih dahulu.
    combined_vectors = np.vstack(preferensi_vektor)
    user_profile = np.mean(combined_vectors, axis=0, keepdims=True)
    
    # Hitung cosine similarity antara user_profile dan seluruh tfidf_matrix (konversikan tfidf_matrix ke dense)
    cosine_scores = cosine_similarity(user_profile, tfidf_matrix.toarray()).flatten()
    
    # Urutkan indeks berdasarkan similarity, descending
    rekomendasi_idx = cosine_scores.argsort()[::-1]
    
    # menghindari jika ada preferensi nama
    if preferensi_nama:
        idx_nama = df[df["name"].isin(preferensi_nama)].index.tolist()
        rekomendasi_idx = [i for i in rekomendasi_idx if i not in idx_nama]
    
    # Ambil top_n rekomendasi
    rekomendasi_items = df.iloc[rekomendasi_idx][:top_n][["name", "category", "location", "rating"]]
    
    return rekomendasi_items

## Try CBF

In [213]:
preferensi_nama = ["Saung Angklung Udjo", "Tebing Keraton"]
preferensi_kategori = ["Alam", "pegunungan", "Kuliner", "Taman Kota"]

rekomendasi_hasil = rekomendasi(
    preferensi_nama=preferensi_nama,
    preferensi_kategori=preferensi_kategori,
    df=wisata, 
    tfidf_matrix=tfidf_matrix,
    vectorizer=vectorizer,
    top_n=10
)

hasil = pd.DataFrame(rekomendasi_hasil)
hasil

,name,category,location,rating
71,Kampung Wisata Sindangbarang,Budaya,"Bogor, Jawa Barat",4.4
80,Alun-Alun Kota Garut,Taman Kota,"Garut, Jawa Barat",4.3
61,Taman Balai Kota Bandung,Taman Kota,"Bandung, Jawa Barat",4.4
73,Dago Dreampark,Taman Hiburan,"Bandung, Jawa Barat",4.5
53,Alun-Alun Bandung,Taman Kota,"Bandung, Jawa Barat",4.4
62,Gunung Ciremai,Pegunungan,"Kuningan, Jawa Barat",4.7
25,Warung Lela,Kuliner,"Bandung, Jawa Barat",4.5
4,Kampung Daun,Kuliner,"Bandung, Jawa Barat",4.5
64,Kampung Naga,Budaya,"Tasikmalaya, Jawa Barat",4.6
19,Green Canyon,Alam,"Pangandaran, Jawa Barat",4.8
